In [1]:
import numpy as np
import pandas as pd
import os
import shutil 
from tqdm import tqdm
import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import time
import copy
from torchvision import transforms, models

In [2]:
train_dir = 'train'
val_dir = 'val'

class_names = ['0', '1']
data_root = 'data'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

for class_name in class_names:
    source_dir = os.path.join(data_root, class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if i % 6 != 0:
            dest_dir = os.path.join(train_dir, class_name) 
        else:
            dest_dir = os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

In [3]:
train_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms)
val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)

batch_size = 16
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
def train_model(model, loss, optimizer, scheduler, num_epochs):
    loss_history = {'train': list(), 'val': list()}
    acc_history = {'train': list(), 'val': list()}
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_dataloader
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                dataloader = val_dataloader
                model.eval()   # Set model to evaluate mode

            running_loss = 0.
            running_acc = 0.

            # Iterate over data.
            for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # forward and backward
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim=1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()

                # statistics
                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean()

            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)
            
            loss_history[phase].append(epoch_loss)
            acc_history[phase].append(epoch_acc)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)
            
        if (epoch + 1) % 5 == 0:
            torch.save(model, 'trained_model')

            history = pd.DataFrame.from_dict({'loss': loss_history['train'], 'acc': acc_history['train']})
            history.to_csv('train_history.csv')
            
            history = pd.DataFrame.from_dict({'loss': loss_history['val'], 'acc': acc_history['val']})
            history.to_csv('val_history.csv')
                
    torch.save(model, 'trained_model')

    history = pd.DataFrame.from_dict({'loss': loss_history['train'], 'acc': acc_history['train']})
    history.to_csv('train_history.csv')

    history = pd.DataFrame.from_dict({'loss': loss_history['val'], 'acc': acc_history['val']})
    history.to_csv('val_history.csv')

    return loss_history, acc_history, model

In [ ]:
model = models.resnet34(pretrained=True)

model.fc = torch.nn.Linear(model.fc.in_features, 2)

model = model.to(device)

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-3)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
loss_history, acc_history, model = train_model(model, loss, optimizer, scheduler, num_epochs=100);

In [ ]:
test_dir = 'test'
shutil.copytree(os.path.join(data_root, 'test'), os.path.join(test_dir, 'unknown'))

In [4]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        name = path.split(os.path.sep)[-1]
        tuple_with_name = (original_tuple + (name,))
        return tuple_with_name
    
test_dataset = ImageFolderWithPaths('test',val_transforms)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [6]:
model = torch.load('trained_model')
model.eval()

test_predictions = []
test_img_paths = []
for inputs, labels, names in tqdm(test_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        preds = model(inputs)
    test_predictions.append(
        torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
    test_img_paths.extend(names)
    
test_predictions = np.concatenate(test_predictions)

100%|██████████| 625/625 [03:54<00:00,  2.45it/s]


In [7]:
def show_input(input_tensor, title=''):
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = input_tensor.permute(1, 2, 0).numpy()
    image = std * image + mean
    plt.imshow(image.clip(0, 1))
    plt.title(title)
    plt.show()
    plt.pause(0.001)

In [ ]:
it = iter(test_dataloader)
for i in range(4):
    inputs, labels, paths = next(it)

    for img, pred in zip(inputs, test_predictions):
        show_input(img, title=pred)

In [19]:
answer = pd.DataFrame.from_dict({'name': test_img_paths, 'prediction': test_predictions})

answer.to_csv('answer.csv', index=False)